# Iterator Pattern

## Intent
Provide a way to access elements of an aggregate object sequentially without exposing its underlying representation.

## Problem
You need to traverse a collection but:
- Don't want to expose internal structure
- Need different traversal algorithms
- Want uniform interface for different collections
- Multiple traversals at same time

**Real-world analogy**: Museum guide (iterator) shows you artifacts (elements) without revealing museum layout (internal structure)

## When to Use
✅ **Use when:**
- Need to traverse complex data structure
- Want to hide internal structure
- Need multiple traversals simultaneously
- Want different traversal algorithms

❌ **Avoid when:**
- Simple list/array is sufficient
- Only one traversal method needed
- Direct access is required

## Pattern Structure
```
┌──────────┐         ┌──────────┐
│ Iterator │         │Aggregate │
├──────────┤         ├──────────┤
│ next()   │         │iterator()│
│hasNext() │         └──────────┘
└──────────┘              ▲
     ▲                    │
     │              ┌─────┴──────┐
┌────┴────┐    ┌───┴────┐       │
│Concrete │◄───│Concrete│       │
│Iterator │    │Aggregat│       │
└─────────┘    └────────┘       │
```

## Python's Built-in Iterator Protocol

Python has native iterator support:
- `__iter__()`: Returns iterator object
- `__next__()`: Returns next element
- `StopIteration`: Signals end of iteration

## Example 1: Without Iterator

**Problem**: Exposing internal structure

In [ ]:
# WITHOUT Iterator - Exposing internals
class BookCollection:
    def __init__(self):
        self.books = []  # Internal structure exposed!
    
    def add_book(self, book):
        self.books.append(book)

# Client needs to know internal structure
collection = BookCollection()
collection.add_book("1984")
collection.add_book("Brave New World")

# Direct access to internal list
for book in collection.books:  # Coupled to implementation!
    print(book)

print("\n❌ Exposes internal structure!")
print("❌ If we change from list to tree, all client code breaks!")

## Implementation: Iterator Pattern (Pythonic Way)

In [ ]:
from abc import ABC, abstractmethod
from typing import Any, List

# Collection with iterator
class BookCollection:
    """Collection that provides iterator."""
    
    def __init__(self):
        self._books: List[str] = []  # Private!
    
    def add_book(self, book: str) -> None:
        self._books.append(book)
    
    def __iter__(self):
        """Return iterator object."""
        return BookIterator(self._books)


# Iterator
class BookIterator:
    """Iterator for book collection."""
    
    def __init__(self, books: List[str]):
        self._books = books
        self._index = 0
    
    def __iter__(self):
        """Iterator must return itself."""
        return self
    
    def __next__(self) -> str:
        """Return next element or raise StopIteration."""
        if self._index < len(self._books):
            book = self._books[self._index]
            self._index += 1
            return book
        raise StopIteration


# Demo
print("=== Iterator Pattern (Pythonic) ===")

collection = BookCollection()
collection.add_book("1984")
collection.add_book("Brave New World")
collection.add_book("Fahrenheit 451")

print("\n📚 Iterating through books:")
for book in collection:  # Works with for loop!
    print(f"  📖 {book}")

print("\n✅ Internal structure is hidden!")
print("✅ Can change implementation without breaking clients!")

## Real-World Example: Social Network Iterator

In [ ]:
from typing import Dict, Set

# User profile
class Profile:
    def __init__(self, user_id: str, name: str, email: str):
        self.user_id = user_id
        self.name = name
        self.email = email
    
    def __repr__(self):
        return f"{self.name} ({self.email})"


# Iterator interface
class ProfileIterator(ABC):
    """Abstract iterator for profiles."""
    
    @abstractmethod
    def __iter__(self):
        pass
    
    @abstractmethod
    def __next__(self) -> Profile:
        pass


# Concrete iterator: Friends
class FriendsIterator(ProfileIterator):
    """Iterate through direct friends."""
    
    def __init__(self, network: 'SocialNetwork', user_id: str):
        self._network = network
        self._user_id = user_id
        self._current = 0
        # Get friend IDs
        self._friends = list(network._friends.get(user_id, set()))
    
    def __iter__(self):
        return self
    
    def __next__(self) -> Profile:
        if self._current < len(self._friends):
            friend_id = self._friends[self._current]
            self._current += 1
            return self._network._profiles[friend_id]
        raise StopIteration


# Concrete iterator: Friends of Friends
class FriendsOfFriendsIterator(ProfileIterator):
    """Iterate through friends of friends."""
    
    def __init__(self, network: 'SocialNetwork', user_id: str):
        self._network = network
        self._user_id = user_id
        self._visited = {user_id}
        self._queue = []
        
        # BFS: get friends of friends
        for friend_id in network._friends.get(user_id, set()):
            self._visited.add(friend_id)
            for fof_id in network._friends.get(friend_id, set()):
                if fof_id not in self._visited:
                    self._queue.append(fof_id)
                    self._visited.add(fof_id)
        
        self._current = 0
    
    def __iter__(self):
        return self
    
    def __next__(self) -> Profile:
        if self._current < len(self._queue):
            fof_id = self._queue[self._current]
            self._current += 1
            return self._network._profiles[fof_id]
        raise StopIteration


# Aggregate: Social Network
class SocialNetwork:
    """Social network with multiple iteration strategies."""
    
    def __init__(self):
        self._profiles: Dict[str, Profile] = {}
        self._friends: Dict[str, Set[str]] = {}
    
    def add_profile(self, profile: Profile) -> None:
        self._profiles[profile.user_id] = profile
        if profile.user_id not in self._friends:
            self._friends[profile.user_id] = set()
    
    def add_friendship(self, user_id1: str, user_id2: str) -> None:
        self._friends[user_id1].add(user_id2)
        self._friends[user_id2].add(user_id1)
    
    def friends(self, user_id: str) -> FriendsIterator:
        """Iterate through direct friends."""
        return FriendsIterator(self, user_id)
    
    def friends_of_friends(self, user_id: str) -> FriendsOfFriendsIterator:
        """Iterate through friends of friends."""
        return FriendsOfFriendsIterator(self, user_id)


# Demo
print("\n=== Social Network Iterator ===")

network = SocialNetwork()

# Add profiles
alice = Profile("1", "Alice", "alice@example.com")
bob = Profile("2", "Bob", "bob@example.com")
charlie = Profile("3", "Charlie", "charlie@example.com")
diana = Profile("4", "Diana", "diana@example.com")
eve = Profile("5", "Eve", "eve@example.com")

for profile in [alice, bob, charlie, diana, eve]:
    network.add_profile(profile)

# Create friendships
network.add_friendship("1", "2")  # Alice - Bob
network.add_friendship("1", "3")  # Alice - Charlie
network.add_friendship("2", "4")  # Bob - Diana
network.add_friendship("3", "5")  # Charlie - Eve

# Iterate through Alice's friends
print("\n👥 Alice's friends:")
for friend in network.friends("1"):
    print(f"  • {friend}")

# Iterate through Alice's friends of friends
print("\n👥 Alice's friends of friends:")
for fof in network.friends_of_friends("1"):
    print(f"  • {fof}")

print("\n✅ Multiple iterators for different traversals!")

## Real-World Example: Tree Traversal Iterators

In [ ]:
# Tree node
class TreeNode:
    def __init__(self, value: int):
        self.value = value
        self.left: TreeNode = None
        self.right: TreeNode = None


# In-order iterator
class InOrderIterator:
    """In-order traversal: Left → Root → Right"""
    
    def __init__(self, root: TreeNode):
        self._stack = []
        self._push_left(root)
    
    def _push_left(self, node: TreeNode) -> None:
        while node:
            self._stack.append(node)
            node = node.left
    
    def __iter__(self):
        return self
    
    def __next__(self) -> int:
        if not self._stack:
            raise StopIteration
        
        node = self._stack.pop()
        self._push_left(node.right)
        return node.value


# Pre-order iterator
class PreOrderIterator:
    """Pre-order traversal: Root → Left → Right"""
    
    def __init__(self, root: TreeNode):
        self._stack = [root] if root else []
    
    def __iter__(self):
        return self
    
    def __next__(self) -> int:
        if not self._stack:
            raise StopIteration
        
        node = self._stack.pop()
        
        # Push right first (so left is processed first)
        if node.right:
            self._stack.append(node.right)
        if node.left:
            self._stack.append(node.left)
        
        return node.value


# Level-order iterator (BFS)
class LevelOrderIterator:
    """Level-order traversal: Level by level, left to right"""
    
    def __init__(self, root: TreeNode):
        self._queue = [root] if root else []
    
    def __iter__(self):
        return self
    
    def __next__(self) -> int:
        if not self._queue:
            raise StopIteration
        
        node = self._queue.pop(0)
        
        if node.left:
            self._queue.append(node.left)
        if node.right:
            self._queue.append(node.right)
        
        return node.value


# Binary tree with multiple iterators
class BinaryTree:
    """Binary tree with multiple traversal methods."""
    
    def __init__(self, root: TreeNode):
        self.root = root
    
    def in_order(self):
        return InOrderIterator(self.root)
    
    def pre_order(self):
        return PreOrderIterator(self.root)
    
    def level_order(self):
        return LevelOrderIterator(self.root)


# Demo
print("\n=== Tree Traversal Iterators ===")

# Build tree:
#       4
#      / \\
#     2   6
#    / \\ / \\
#   1  3 5  7
root = TreeNode(4)
root.left = TreeNode(2)
root.right = TreeNode(6)
root.left.left = TreeNode(1)
root.left.right = TreeNode(3)
root.right.left = TreeNode(5)
root.right.right = TreeNode(7)

tree = BinaryTree(root)

print("\n🌳 Tree structure:")
print("       4")
print("      / \\")
print("     2   6")
print("    / \\ / \\")
print("   1  3 5  7")

print("\n📍 In-order (Left → Root → Right):")
print("  ", list(tree.in_order()))

print("\n📍 Pre-order (Root → Left → Right):")
print("  ", list(tree.pre_order()))

print("\n📍 Level-order (BFS):")
print("  ", list(tree.level_order()))

print("\n✅ Same tree, different iteration strategies!")

## Python Generator as Iterator

In [ ]:
# Using generators (more Pythonic!)
class Range:
    """Custom range using generator."""
    
    def __init__(self, start: int, end: int):
        self.start = start
        self.end = end
    
    def __iter__(self):
        """Generator function - simpler than creating iterator class!"""
        current = self.start
        while current < self.end:
            yield current
            current += 1


# Tree traversal with generators
class SimpleTree:
    def __init__(self, root: TreeNode):
        self.root = root
    
    def in_order(self):
        """Generator for in-order traversal."""
        def traverse(node):
            if node:
                yield from traverse(node.left)
                yield node.value
                yield from traverse(node.right)
        
        return traverse(self.root)
    
    def pre_order(self):
        """Generator for pre-order traversal."""
        def traverse(node):
            if node:
                yield node.value
                yield from traverse(node.left)
                yield from traverse(node.right)
        
        return traverse(self.root)


# Demo
print("\n=== Generators as Iterators ===")

print("\n1. Custom Range:")
for i in Range(1, 5):
    print(f"  {i}", end=" ")

print("\n\n2. Tree traversal with generators:")
tree = SimpleTree(root)
print("  In-order:", list(tree.in_order()))
print("  Pre-order:", list(tree.pre_order()))

print("\n✅ Generators are often more Pythonic than iterator classes!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Single Responsibility**: Separates traversal from collection
2. **Open/Closed Principle**: Add new iterators without modifying collection
3. **Multiple traversals**: Iterate same collection differently
4. **Concurrent iteration**: Multiple iterators simultaneously
5. **Lazy evaluation**: Generate elements on demand

### ❌ Disadvantages
1. **Overkill**: Simple collections don't need it
2. **Less efficient**: Than direct access for simple structures
3. **More classes**: Additional iterator classes

## External vs Internal Iterator

**External Iterator** (what we've shown):
- Client controls iteration
- Can pause/resume
- More flexible

```python
it = collection.iterator()
while it.has_next():
    item = it.next()
```

**Internal Iterator**:
- Collection controls iteration
- Simpler for client
- Less flexible

```python
collection.for_each(lambda item: print(item))
```

## Common Use Cases

1. **Collections**: Lists, trees, graphs
2. **File systems**: Directory traversal
3. **Networks**: Graph traversal
4. **Social networks**: Friends, followers
5. **Menu systems**: Navigation
6. **Playlists**: Song iteration
7. **Search results**: Paginated results

## Related Patterns

- **Composite**: Often used with Iterator to traverse tree structures
- **Factory Method**: Create appropriate iterator
- **Memento**: Store iteration state

## Best Practices

1. **Use generators**: In Python, prefer generators over iterator classes
2. **Follow protocol**: Implement `__iter__` and `__next__`
3. **Raise StopIteration**: Signal end of iteration
4. **Multiple iterators**: Allow concurrent iteration
5. **Snapshot vs live**: Decide if iterator reflects changes
6. **Immutable during iteration**: Document if collection shouldn't change

## Summary

Iterator pattern enables:
- Sequential access without exposing structure
- Multiple traversal algorithms
- Uniform interface for different collections
- Concurrent iteration

Perfect for: Collections, trees, graphs, file systems, social networks.

**Key Insight**: Separate traversal logic from collection structure, providing uniform access interface!